In [3]:
from custom_widgets import POPUP_TEMPLATE
from studydata import NerrStudyData

import folium
from folium.plugins import MarkerCluster
from folium import IFrame

nerr = NerrStudyData(online_source=False)
stations_df = nerr.get_stations()

In [8]:
stations_df

,StationCode,StationName,Latitude,Longitude,Status,ActiveDates,State,ReserveName,GMTOffset,StationType,geometry
137,job06wq,Station 6,17.934986,-66.227933,Inactive,Dec 1996-Dec 1998,pr,Jobos Bay,-4,Water Quality,POINT (-66.22793 17.93499)
138,job09nut,Station 9,17.943056,-66.238583,Active,Jul 2002-,pr,Jobos Bay,-4,Nutrients,POINT (-66.23858 17.94306)
139,job09wq,Station 9,17.943056,-66.238583,Active,Dec 1995-,pr,Jobos Bay,-4,Water Quality,POINT (-66.23858 17.94306)
140,job10nut,Station 10,17.938611,-66.257736,Active,Jul 2002-,pr,Jobos Bay,-4,Nutrients,POINT (-66.25774 17.93861)
141,job10wq,Station 10,17.938611,-66.257736,Active,Feb 1996-,pr,Jobos Bay,-4,Water Quality,POINT (-66.25774 17.93861)
142,job11wq,Station 11,17.944608,-66.262028,Inactive,Mar 1996-Dec 1998,pr,Jobos Bay,-4,Water Quality,POINT (-66.26203 17.94461)
143,job18wq,Station 18,17.944433,-66.231881,Inactive,Oct 1998- Nov 1998,pr,Jobos Bay,-4,Water Quality,POINT (-66.23188 17.94443)
144,job19nut,Station 19,17.942914,-66.228825,Active,Jul 2002-,pr,Jobos Bay,-4,Nutrients,POINT (-66.22883 17.94291)
145,job19wq,Station 19,17.942914,-66.228825,Active,Apr 2002-,pr,Jobos Bay,-4,Water Quality,POINT (-66.22883 17.94291)
146,job20nut,Station 20,17.930317,-66.211472,Active,Jul 2002-,pr,Jobos Bay,-4,Nutrients,POINT (-66.21147 17.93032)


In [78]:
# Follium code...

import folium
from folium.plugins import MarkerCluster
from folium import IFrame, Map, Popup, Icon, Marker, LayerControl
import ipywidgets as widgets

class StudyMap():

    PR_CENTER = [17.95524, -66.2200]

    @staticmethod
    def create_map(center=PR_CENTER, zoom=13, *args, **kwargs):
        m1 = Map(
            location=center,
            zoom_start=zoom,
            *args,
            **kwargs
        )
        return m1

    # Based on the popup html...
    @staticmethod
    def create_popups_from_template(
        location,
        html_template=POPUP_TEMPLATE,
        *args,
        **kwargs
    ):
        iframe = IFrame(
            html=html_template(*args, **kwargs),
            width=400,
            height=200
        )
        popup = Popup(
            iframe,
            max_width=500
        )
        return popup

    # Appends markers to map based on df
    # and return map
    @staticmethod
    def create_stations_map(df):
        m1 = StudyMap.create_map()

        for station_type in df.StationType.unique().tolist():
            mask = (df["StationType"] == station_type)
            df2 = df.loc[mask]

            # markers that will be added to the cluster...
#             marker_cluster = MarkerCluster(name=station_type).add_to(m1)
            for idx, row in df2.iterrows():
                location = [row.geometry.y, row.geometry.x]
                popup = StudyMap.create_popups_from_template(
                    location,
                    POPUP_TEMPLATE,
                    row.StationName,
                    row.StationCode,
                    row.StationType,
                    row.Status,
                    row.ReserveName,
                    row.ActiveDates,
                    row.StationType
                )

                # Give color to marker based on station_type
                def get_color(station_type):
                    switcher = {
                        "Meteorological": "blue",
                        "Nutrients": "green",
                        "Water Quality": "red",
                    }

                    color = switcher.get(station_type, "gray")
                    return color

#                 icon_name = "check-circle" if row.Status == "Active" else "times-circle"
                icon_name = "glyphicon-ok" if row.Status == "Active" else "glyphicon-remove"

                icon = Icon(
                    icon=icon_name,
                    color=get_color(row.StationType),
                    icon_color='white',
                    spin=False
                )

                Marker(
                    location=location,
                    popup=popup,
                    title=f"{row.StationType} - {row.StationCode}",
                    icon=icon,
                    draggable=False,
                    rise_on_hover=True
                ).add_to(m1)
            
        # Create the layer control and add to map
        control = LayerControl(position='topright')
        control.add_to(m1)
        return m1

In [79]:
StudyMap.create_stations_map(stations_df)